# YOUR PROJECT TITLE

Imports and set magics:

In [222]:
# First, we import an API from DST, specifying the language
import pydst                          
Dst = pydst.Dst(lang='en')            
# Next, the packages for data analysis are imported
import pandas as pd                   
import numpy as np                    
import matplotlib.pyplot as plt       
import ipywidgets as widgets          

# Autoreloading modules when code is run
%load_ext autoreload
%autoreload 2

# Local modules
import dataproject


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Read and clean data

**Read the  data** from DSTI. We want to analyze the table named NKN2 including the Danish real gross national disposable income (etc.) by transaction and price unit:

In [223]:
Dst.get_data(table_id = 'NKN2')       # Retrieves NKN2


#Import stok data from IEX
#start = datetime(2014,1,1)
#end   = datetime(2019,12,31)
#from pandas_datareader import data as wb
#pg=wb.DataReader('PG',data_source='yahoo',start='2019-1-1')
#plt.plot(pg)
#
#firms = [] 
#for i, stock_name in enumerate(['FB','AAPL', 'GOOGL', 'NFLX', 'AMZN', 'SPY']): 
 #    firm_stock = pandas_datareader.iex.daily.IEXDailyReader(stock_name, start, end).read()
  #   firms = pd.datareader.iex.daily.IEXDailyReader(stock_name, start, end).read()
  #   firm_stock['firm'] = stock_name
   #  firms.append(firm_stock)

#stocks = pd.concat(firms)

# a. load
#empl = pd.read_excel('RAS200.xlsx', skiprows=2)

# b. drop columns
#drop_these = ['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3']
#empl.drop(drop_these, axis=1, inplace=True)

# c. rename columns
#empl.rename(columns = {'Unnamed: 4':'municipality'}, inplace=True)

,TRANSAKT,PRISENHED,SÆSON,TID,INDHOLD
0,B.1*g Gross domestic product,Current prices,Non-seasonally adjusted,1990Q1,210169


In [224]:
Dst.get_variables(table_id = 'NKN1')  # Retrieves variables of NKN1

,id,text,elimination,time,values
0,TRANSAKT,transaction,False,False,"[{'id': 'B1GQK', 'text': 'B.1*g Gross domestic..."
1,PRISENHED,price unit,False,False,"[{'id': 'V_M', 'text': 'Current prices, (bill...."
2,SÆSON,seasonal adjustment,False,False,"[{'id': 'N', 'text': 'Non-seasonally adjusted'..."
3,Tid,time,False,True,"[{'id': '1990K1', 'text': '1990Q1'}, {'id': '1..."


In [225]:

###Here, we wish to extracting the seasonally adjusted GDP in chained 2010-prices (bill. DKK) for the period 1990Q1 - 2019Q4.

df = Dst.get_data(table_id = 'NKN1',
     variables = {'TRANSAKT':['B1GQK'],
     'PRISENHED':['LKV_M'],
     'SÆSON':['Y'],'TID':['*']})        # Extracts the desired variables
pd.concat([df.head(3), df.tail(3)])     # Shows first three and last three rows

,TRANSAKT,PRISENHED,SÆSON,TID,INDHOLD
0,B.1*g Gross domestic product,"2010-prices, chained values, (bill. DKK.)",Seasonally adjusted,1990Q1,..
1,B.1*g Gross domestic product,"2010-prices, chained values, (bill. DKK.)",Seasonally adjusted,1990Q2,..
2,B.1*g Gross domestic product,"2010-prices, chained values, (bill. DKK.)",Seasonally adjusted,1990Q3,..
117,B.1*g Gross domestic product,"2010-prices, chained values, (bill. DKK.)",Seasonally adjusted,2019Q2,532.5
118,B.1*g Gross domestic product,"2010-prices, chained values, (bill. DKK.)",Seasonally adjusted,2019Q3,535.1
119,B.1*g Gross domestic product,"2010-prices, chained values, (bill. DKK.)",Seasonally adjusted,2019Q4,538.1


In [226]:
#We now remove redundant variables, so only time and GDP remain.

df1 = df.drop('TRANSAKT',1).drop('PRISENHED',1).drop('SÆSON',1) # Removes TRANSAKT, PRISENHED and SÆSON
pd.concat([df1.head(5), df1.tail(5)])                           # Shows first five and last five rows

,TID,INDHOLD
0,1990Q1,..
1,1990Q2,..
2,1990Q3,..
3,1990Q4,..
4,1991Q1,326.6
115,2018Q4,526.3
116,2019Q1,526.2
117,2019Q2,532.5
118,2019Q3,535.1
119,2019Q4,538.1


In [227]:
#We then extract time and GDP as arrays. We also remove empty cells in GDP and convert this variable to the type of float.

time = df1['TID']                       # Extracts time
gdp  = df1['INDHOLD']                   # Extratcs GDP

gdp.replace('..',np.nan, inplace=True)
gdp.dropna(inplace=True)                # Removes empty cells
gdp = gdp.iloc[:].astype(float)         # Converts to float

gdp = gdp.reset_index(drop=True)        # Resets index

pd.concat([gdp.head(3), gdp.tail(3)])   # Shows first three and last three rows

0      326.6
1      325.4
2      329.5
113    532.5
114    535.1
115    538.1
Name: INDHOLD, dtype: float64

The dataset now looks like this:

In [228]:
#empl.head()

**Remove all rows which are not municipalities**:

In [229]:
#empl = dataproject.only_keep_municipalities(empl)
#empl.head()

**Convert the dataset to long format**:

In [230]:
# a. rename year columns
#mydict = {str(i):f'employment{i}' for i in range(2008,2018)}
#empl.rename(columns = mydict, inplace=True)

# b. convert to long
#empl_long = pd.wide_to_long(empl, stubnames='employment', i='municipality', j='year').reset_index()

# c. show
#empl_long.head()

## Income data

**Read the income data** in ``INDKP101.xlsx`` and **clean it**:

In [231]:
# a. load
#inc = pd.read_excel('INDKP101.xlsx', skiprows=2)

# b. drop and rename columns
#inc.drop([f'Unnamed: {i}' for i in range(3)], axis=1, inplace=True)
#inc.rename(columns = {'Unnamed: 3':'municipality'}, inplace=True)

# c. drop rows with missing
#inc.dropna(inplace=True)

# d. remove non-municipalities
#inc = dataproject.only_keep_municipalities(inc)

# e. convert to long
#inc.rename(columns = {str(i):f'income{i}' for i in range(1986,2018)}, inplace=True)
#inc_long = pd.wide_to_long(inc, stubnames='income', i='municipality', j='year').reset_index()

# f. show
#inc_long.head(5)

> **Note:** The function ``dataproject.only_keep_municipalities()`` is used on both the employment and the income datasets.

## Explore data set

In order to be able to **explore the raw data**, we here provide an **interactive plot** to show, respectively, the employment and income level in each municipality

The **static plot** is:

In [232]:
#def plot_empl_inc(empl,inc,dataset,municipality): 
    
 #   if dataset == 'Employment':
  #      df = empl
   #     y = 'employment'
    #else:
     #   df = inc
      #  y = 'income'
    
   # I = df['municipality'] == municipality
   # ax = df.loc[I,:].plot(x='year', y=y, style='-o')

The **interactive plot** is:

In [233]:
#widgets.interact(plot_empl_inc, 
    
 #   empl = widgets.fixed(empl_long),
  #  inc = widgets.fixed(inc_long),
   # dataset = widgets.Dropdown(description='Dataset', 
    #                           options=['Employment','Income']),
    #municipality = widgets.Dropdown(description='Municipality', 
     #                               options=empl_long.municipality.unique())
                 
#); 

ADD SOMETHING HERE IF THE READER SHOULD KNOW THAT E.G. SOME MUNICIPALITY IS SPECIAL.

# Merge data sets

We now create a data set with **municpalities which are in both of our data sets**. We can illustrate this **merge** as:

In [234]:
#plt.figure(figsize=(15,7))
#v = venn2(subsets = (4, 4, 10), set_labels = ('inc', 'empl'))
#v.get_label_by_id('100').set_text('dropped')
#v.get_label_by_id('010').set_text('dropped' )
#v.get_label_by_id('110').set_text('included')
#plt.show()

In [235]:
#merged = pd.merge(empl_long, inc_long, how='inner',on=['municipality','year'])

#print(f'Number of municipalities = {len(merged.municipality.unique())}')
#print(f'Number of years          = {len(merged.year.unique())}')

# Analysis

To get a quick overview of the data, we show some **summary statistics by year**:

In [236]:
#merged.groupby('year').agg(['mean','std']).round(2)

ADD FURTHER ANALYSIS. EXPLAIN THE CODE BRIEFLY AND SUMMARIZE THE RESULTS.

# Conclusion

ADD CONCISE CONLUSION.